In [8]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import calendar
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from numpy import array
from math import sqrt
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from matplotlib import pyplot as plt
from matplotlib.patches import Patch
import keras.backend as K
import itertools
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from random import random
#!pip install pmdarima --quiet
import pmdarima as pm

# Code adapted from https://medium.com/data-science/time-series-forecasting-with-arima-sarima-and-sarimax-ee61099e78f6
# Plot data to view
def plot_data(df, feature):
    plt.figure(figsize=(15,7))
    plt.title(str(feature)+" by Month")
    plt.xlabel('Month')
    plt.ylabel(str(feature))
    plt.plot(df)
    plt.show()

#Determine rolling statistics to find trends
def rolling_statistics(df):
    df["rolling_avg"] = df.rolling(window=12).mean() #window size 12 denotes 12 months, giving rolling mean at yearly level
    df["rolling_std"] = df.rolling(window=12).std()

    #Plot rolling statistics
    plt.figure(figsize=(15,7))
    plt.plot(df, color='#379BDB', label='Original')
    plt.plot(df["rolling_avg"], color='#D22A0D', label='Rolling Mean')
    plt.plot(df["rolling_std"], color='#142039', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)

#Augmented Dickey-Fuller Test to test if the time series is stationary
#If ADF has p <= 0.05, data are stationary
def ADF(df):
    print('Results of Dickey Fuller Test for temperature:')
    dftest = adfuller(df, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value

    print(dfoutput)

    return dfoutput

#Standard ARIMA Model
def fit_ARIMA_model(df):
    model = pm.auto_arima(df, start_p=1, start_q=1, test='adf', # use adftest to find optimal 'd' 
                          max_p=12, max_q=12, # maximum p and q
                          m=1, # frequency of series (if m==1, seasonal is set to FALSE automatically)
                          d=None,# let model determine 'd'
                          seasonal=False, # No Seasonality for standard ARIMA
                          trace=False, #logs 
                          error_action='warn', #shows errors ('ignore' silences these)
                          suppress_warnings=True,
                          stepwise=True
                         )
    return model

# SARIMAX Model
def fit_SARIMAX_model(df, exog):
    model = pm.auto_arima(df, start_p=1, start_q=1, exogenous=exog,
                         test='adf', # use adftest to find optimal 'd'
                         max_p=12, max_q=12, m=12, #12 is the frequency of the cycle
                         start_P=0, seasonal=True,
                         d=None, D=1, 
                         trace=False,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

    return model

def plot_diagnostics(model):
    model.plot_diagnostics(figsize=(15,12))
    plt.show
    
def ARIMA_forecast(ARIMA_model, df, periods):
    # Forecast
    n_periods = periods
    fitted, confint = ARIMA_model.predict(n_periods=n_periods, return_conf_int=True)
    index_of_fc = pd.date_range(df.index[-1] + pd.DateOffset(months=1), periods = n_periods, freq='MS')

    # make series for plotting purpose
    fitted_series = pd.Series(fitted, index=index_of_fc)
    lower_series = pd.Series(confint[:, 0], index=index_of_fc)
    upper_series = pd.Series(confint[:, 1], index=index_of_fc)

    # Plot
    plt.figure(figsize=(15,7))
    plt.plot(df, color='#1f76b4')
    plt.plot(fitted_series, color='darkgreen')
    plt.fill_between(lower_series.index, 
                    lower_series, 
                    upper_series, 
                    color='k', alpha=.15)

    plt.title("ARIMA Forecast")
    plt.show()

    return fitted_series, lower_series, upper_series

def SARIMAX_forecast(SARIMAX_model, df, exog, periods):
    # Forecast
    n_periods = periods

    forecast_df = pd.DataFrame({"month":pd.date_range(df.index[-1], periods = n_periods, freq='MS').month},
                    index = pd.date_range(df.index[-1] + pd.DateOffset(months=1), periods = n_periods, freq='MS'))

    fitted, confint = SARIMAX_model.predict(n_periods=n_periods, 
                                            return_conf_int=True,
                                            exogenous=exog)
    index_of_fc = pd.date_range(df.index[-1] + pd.DateOffset(months=1), periods = n_periods, freq='MS')

    # make series for plotting purpose
    fitted_series = pd.Series(fitted, index=index_of_fc)
    lower_series = pd.Series(confint[:, 0], index=index_of_fc)
    upper_series = pd.Series(confint[:, 1], index=index_of_fc)

    # Plot
    plt.figure(figsize=(15,7))
    plt.plot(df, color='#1f76b4')
    plt.plot(fitted_series, color='darkgreen')
    plt.fill_between(lower_series.index, 
                    lower_series, 
                    upper_series, 
                    color='k', alpha=.15)

    plt.title("SARIMAX Forecast")
    plt.show()

    return fitted_series, lower_series, upper_series

# date-time parsing function for loading the dataset
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')
    
def rmse (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))
    
def mape (y_true, y_pred):
    return 100*K.mean(K.sqrt(K.square(y_true - y_pred))/y_true)
    
def pearson (y_true, y_pred):
    return (K.square(K.mean((y_true - K.mean(y_true))*(y_pred - K.mean(y_pred)))))/(K.mean(K.square(y_true - K.mean(y_true)))*K.mean(K.square(y_pred - K.mean(y_pred))))

# create a differenced series
#Taken from: https://machinelearningmastery.com/multi-step-time-series-forecasting-long-short-term-memory-networks-python/
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff

# transform series into training sets
def prepare_training_data(data, n_lag, n_seq, n_time_steps):
    
    #Prepare data for time series forecasting.
        
    #Parameters:
    #x (array-like): Input features.
    #y (array-like): Target values.
    #n_test (int): Number of test samples (rows).
    #n_lag (int): Number of lag observations.
    #n_seq (int): Number of sequence observations.
    #n_train (int): Number of training samples (rows).
        
    #Returns:
    #tuple: Training and test datasets.
    
    n_vars = len(data[0][0])

    # Each weather station has 227 time steps (the first 180 have no nan values)
    # Loop through data, grabbing one weather station (ws) at a time, 
    # differencing on each ws and separating by training (first 226-n_lag-n_seq-n_test time steps) 
    # and testing (n_test time steps) to scale data on training only.
    # We then recombine the training and testing datasets to change each ws to a supervised learning problem by taking all the first 180 time steps for all 12 predictors
    # and changing these to (t-n_lag) to (t-1) since we lose one row through differencing. We then shift forward only one dependent variable (temperature or specific humidity)
    # for time steps t to (t+n_seq)


    diff_values = []
    
    for ws in range(84):
        
        # transform data to be stationary
        diff_series = difference(data[ws], 1)
        for i in range(len(diff_series)):
            diff_values_row = []
            for j in range(len(diff_series[0])):
                diff_values_row.append(diff_series[i][j])
            diff_values.append(diff_values_row)
    
    # rescale values to 0, 1
    scaler_all_features =  MinMaxScaler(feature_range=(0, 1))
    scaler =  MinMaxScaler(feature_range=(0, 1))
    train_scaled_values = scaler_all_features.fit_transform(diff_values)
    response_train_values = []
    for i in range(len(diff_values)):
        response_train_values.append(diff_values[i][0]) # Uses first column (temperatures) as response variable
    response_train_values = np.array(response_train_values)
    response_train_values = response_train_values.reshape(len(response_train_values), 1)

    # Fit the scaler for just the response variable for use later when forecasting
    response_scaled_values = scaler.fit_transform(response_train_values) 
    scaled_values = scaler_all_features.transform(diff_values)

    train = []

    # Transform each weather station as a separate "batch"
    for ws in range(84):
        # transform into supervised learning problem X, y
        first = (n_time_steps-1)*ws
        last = (n_time_steps-1)*ws+(n_time_steps-2)
        scaled_values_batch = scaled_values[first:last]
        supervised = series_to_supervised(scaled_values_batch, n_lag, n_seq)
        supervised_values = supervised.values
        train.append([supervised_values])
    
    return scaler, scaler_all_features, train

# transform series into testing and validation sets
def prepare_testing_and_validation_data(data, n_lag, n_seq, n_time_steps, scaler_all_features):
    
    #Prepare data for time series forecasting.
        
    #Parameters:
    #x (array-like): Input features.
    #y (array-like): Target values.
    #n_test (int): Number of test samples (rows).
    #n_lag (int): Number of lag observations.
    #n_seq (int): Number of sequence observations.
    #n_train (int): Number of training samples (rows).
        
    #Returns:
    #tuple: Training and test datasets.
    
    n_vars = len(data[0][0])

    # Each weather station has 227 time steps (the first 180 have no nan values)
    # Loop through data, grabbing one weather station (ws) at a time, 
    # differencing on each ws and separating by training (first 226-n_lag-n_seq-n_test time steps) 
    # and testing (n_test time steps) to scale data on training only.
    # We then recombine the training and testing datasets to change each ws to a supervised learning problem by taking all the first 180 time steps for all 12 predictors
    # and changing these to (t-n_lag) to (t-1) since we lose one row through differencing. We then shift forward only one dependent variable (temperature or specific humidity)
    # for time steps t to (t+n_seq)


    diff_values = []
    
    for ws in range(21):
        
        # transform data to be stationary
        diff_series = difference(data[ws], 1)
        for i in range(len(diff_series)):
            diff_values_row = []
            for j in range(len(diff_series[0])):
                diff_values_row.append(diff_series[i][j])
            diff_values.append(diff_values_row)

    # rescale values to 0, 1
    scaled_values = scaler_all_features.transform(diff_values)

    validation = []
    test = []

    # Transform each weather station as a separate "batch"
    for ws in range(21):
        # transform into supervised learning problem X, y
        first = (n_time_steps-1)*ws
        last = (n_time_steps-1)*ws+(n_time_steps-2)
        scaled_values_batch = scaled_values[first:last]
        supervised = series_to_supervised(scaled_values_batch, n_lag, n_seq)
        supervised_values = supervised.values
        # training/test/validation split is 80%/10%/10%
        if ws < 11:
            test.append([supervised_values])
        else:
            validation.append([supervised_values])
    
    return validation, test

def prepare_data(data):

    # prepare data for normalization
    series = Series(data)
    values = series.values
    values = values.reshape((len(values), 1))

    # train the normalization
    scaler = StandardScaler()
    scaler_all_features = StandardScaler()
    scaler = scaler.fit(values)
    print('Mean: %f, StandardDeviation: %f' % (scaler1.mean_, sqrt(scaler1.var_)))
    
    # normalize the dataset and print
    standardized = scaler.transform(values)

    scaled_values = np.array(standardized[:,0])

    return scaler, scaler_all_features, scaled_values 

def plot_kfold(cv, X, y, ax, n_splits, xlim_max=105):
    
    #Plots the indices for a cross-validation object.
    #Taken from https://www.geeksforgeeks.org/cross-validation-using-k-fold-with-scikit-learn/
    
    #Parameters:
    #cv: Cross-validation object
    #X: Feature set
    #y: Target variable
    #ax: Matplotlib axis object
    #n_splits: Number of folds in the cross-validation
    #xlim_max: Maximum limit for the x-axis
        
    # Set color map for the plot
    cmap_cv = plt.cm.coolwarm
    cv_split = cv.split(X=X, y=y)
        
    for i_split, (train_idx, test_idx) in enumerate(cv_split):
        # Create an array of NaNs and fill in training/testing indices
        indices = np.full(len(X), np.nan)
        indices[test_idx], indices[train_idx] = 1, 0
            
        # Plot the training and testing indices
        ax_x = range(len(indices))
        ax_y = [i_split + 0.5] * len(indices)
        ax.scatter(ax_x, ax_y, c=indices, marker="_", 
                   lw=10, cmap=cmap_cv, vmin=-0.2, vmax=1.2)
    
        # Set y-ticks and labels
        y_ticks = np.arange(n_splits) + 0.5
        ax.set(yticks=y_ticks, yticklabels=range(n_splits),
               xlabel="Weather Station index (file_id)", ylabel="Fold",
               ylim=[n_splits, -0.2], xlim=[0, xlim_max])
    
        # Set plot title and create legend
        ax.set_title("KFold", fontsize=14)
        legend_patches = [Patch(color=cmap_cv(0.8), label="Testing set"), 
                          Patch(color=cmap_cv(0.02), label="Training set")]
        ax.legend(handles=legend_patches, loc=(1.03, 0.8))

#Main

#Configure
n_seq = 60
if n_seq > 46:
    n_lag = 179 - n_seq + 46
else:
    n_lag = 179
n_time_steps = 227
n_test = 1

print("Model Parameters:")
print("n_lag (number of input time steps): "+str(n_lag))
print("n_seq (number of output/future prediction time steps): "+str(n_seq))

# Create 2D array with file_ids to use for sample creation
array = np.array([
    6501, 6541, 6640, 6668, 6678, 
    6687, 6697, 6714, 6744, 6772, 
    6783, 6840, 6844, 6854, 6870, 
    6891, 6895, 6899, 6901, 6909, 
    6929, 6950, 6963, 6969, 6994, 
    7032, 7057, 7094, 7095, 7100, 
    7108, 7116, 7119, 7131, 7139, 
    7152, 7155, 7156, 7182, 7193, 
    7202, 7239, 7280, 7286, 7287, 
    7311, 7321, 7329, 7347, 7350, 
    7354, 7357, 7361, 7414, 7423, 
    7424, 7432, 7463, 7482, 7489, 
    7528, 7531, 7534, 7538, 7549, 
    7553, 7555, 7562, 7571, 7573, 
    7574, 7575, 7585, 7599, 7603, 
    7606, 7622, 7652, 7671, 7704, 
    7786, 7805, 7816, 7838, 7861, 
    7862, 7863, 7870, 7892, 7907, 
    7938, 7962, 7979, 7987, 7999, 
    8000, 8034, 8083, 8120, 8133, 
    8184, 8186, 8247, 8248, 9858])

#Create arrays holding the 5-fold cross-validation indices gathered for consistency across models
train_array = []
test_array = []
    
train_array.append([1, 2, 3, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 19, 20, 21, 22, 
                        23, 24, 25, 27, 28, 29, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 
                        43, 44, 46, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 
                        62, 63, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 81, 
                        82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 95, 97, 98, 100, 101, 102, 103])
test_array.append([0, 4, 10, 12, 18, 26, 30, 31, 33, 45, 47, 53, 64, 65, 77, 80, 89, 94, 96, 99, 104])
    
train_array.append([0, 1, 2, 3, 4, 6, 7, 8, 10, 12, 13, 14, 17, 18, 19, 20, 21, 23, 
                        24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 36, 37, 38, 41, 43, 45, 
                        46, 47, 48, 49, 50, 51, 52, 53, 54, 57, 58, 59, 60, 61, 63, 64, 
                        65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 77, 80, 81, 82, 83, 84, 
                        86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 104])
test_array.append([5, 9, 11, 15, 16, 22, 28, 35, 39, 40, 42, 44, 55, 56, 62, 72, 76, 78, 79, 85, 103])
    
train_array.append([0, 1, 2, 4, 5, 9, 10, 11, 12, 14, 15, 16, 18, 20, 21, 22, 23, 26, 
                    28, 29, 30, 31, 32, 33, 35, 36, 37, 39, 40, 41, 42, 44, 45, 46, 47, 
                    48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 
                    70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 82, 83, 85, 86, 87, 88, 89, 
                    90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104])
test_array.append([3, 6, 7, 8, 13, 17, 19, 24, 25, 27, 34, 38, 43, 49, 66, 67, 68, 69, 73, 81, 84])

train_array.append([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 
                        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 
                        35, 37, 38, 39, 40, 42, 43, 44, 45, 47, 49, 51, 52, 53, 55, 56, 
                        60, 62, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 76, 77, 78, 79, 
                        80, 81, 82, 84, 85, 86, 87, 88, 89, 92, 93, 94, 95, 96, 99, 102, 103, 104])
test_array.append([32, 36, 41, 46, 48, 50, 54, 57, 58, 59, 61, 63, 70, 75, 83, 90, 91, 97, 98, 100, 101])
    
train_array.append([0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 22,
                        24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 
                        42, 43, 44, 45, 46, 47, 48, 49, 50, 53, 54, 55, 56, 57, 58, 59, 
                        61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 75, 76, 77, 78, 
                        79, 80, 81, 83, 84, 85, 89, 90, 91, 94, 96, 97, 98, 99, 100, 101, 103, 104])
test_array.append([1, 2, 14, 20, 21, 23, 29, 37, 51, 52, 60, 71, 74, 82, 86, 87, 88, 92, 93, 95, 102])
    
# Equations for three Principal Components from PCA using response variables combined with other predictors
#PC1=-0.0002714X1+0.02612X2+0.03858X3-0.007658X4+0.001592X5-0.02087X6+0.8564X7-0.1468X8+0.01192X9-0.0001049X10+0.01913X11+0.02076X12
#PC2=0.0003944X1+0.002204X2+0.01052X3+0.3248X4-0.0009976X5-0.04421X6+2.3406X7+0.06103X8+0.08841X9+0.00009018X10+0.05678X11-0.002022X12
#PC3=-0.00007998X1-0.0006124X2-0.001063X3-0.01855X4+0.00001956X5+0.01170X6+0.6076X7+0.4664X8-0.002995X9+0.008185X10+0.8815X11-0.0004730X12
    
# Equations for three Principal Components from PCA omitting both response variables,
#PC-1=-0.0004514X1+0.03194X2-0.04343X3+0.002243X4-0.02252X5+0.9877X6-0.2265X7+0.006144X8-0.0001488X9+0.02943X10
#PC-2=0.0001702X1+0.005484X2+0.2057X3-0.0003188X4-0.02584X5+1.6963X6-0.05890X7+0.05809X8+1.9748X9+0.03686X10
#PC-3=-0.00006323X1-0.001180X2-0.02384X3-0.00002833X4+0.01170X5+0.5204X6+0.4791X7-0.004318X8+0.008271X9+0.8765X10

#X1 = slp	
#X2 = wbt	
#X3 = water	
#X4 = GHI
#X5 = WDSP
#X6 = PRCP	
#X7 = SNDP	
#X8 = Region	
#X9 = SA	
#X10 = ONI

# Get the current working directory 
current_directory = os.getcwd() 

# Print the current working directory 
print(current_directory)

# Define the directory containing the files 
path = current_directory+"\\Modeling\\"
print(path)

filename = path + 'Final_Monthly_Dataset.csv'

# load dataset
df = read_csv(filename, header=0, parse_dates=[0], index_col=0, date_format='%Y-%m')
    
df = df.rename(columns={'Unnamed: 0' : 'indices'})
    
#Remove unused columns
df = df.drop(['Day', 'vapor_pressure'], axis=1)

# Round numbers in columns to reasonable precision,
df['temperatures'] = np.round(df['temperatures'], 2)
df['slp'] = np.round(df['slp'], 2)
df['wet_bulb_temperature'] = np.round(df['wet_bulb_temperature'], 2)
df['specific_humidity'] = np.round(df['specific_humidity'], 2)
df['GHI'] = np.round(df['GHI'], 2)
df['PRCP'] = np.round(df['PRCP'], 2)
df['SNDP'] = np.round(df['SNDP'], 2)
df['solar_activity'] = np.round(df['solar_activity'], 2)
df['ONI'] = np.round(df['ONI'], 2)
df['water'] = np.round(df['water'], 0)
df['region'] = np.round(df['region'], 0)
    
df_trimmed = df[df['file_id'] != 7533] # Remove file_id 7533 so there are 105 weather stations for 5-fold CV
df_trimmed = df_trimmed.drop(['Year', 'Month', 'date', 'latitude', 'longitude', 'elevation'], axis=1)

results_df = df_trimmed

# Calculate principal components excluding response variables
results_df['PC1']=-0.0004514*results_df['slp']+0.03194*results_df['wet_bulb_temperature']-0.04343*results_df['water']+0.002243*results_df['GHI']-0.02252*results_df['WDSP']
+0.9877*results_df['PRCP']-0.2265*results_df['SNDP']+0.006144*results_df['region']-0.0001488*results_df['solar_activity']+0.02943*results_df['ONI']
results_df['PC2']=0.0001702*results_df['slp']+0.005484*results_df['wet_bulb_temperature']+0.2057*results_df['water']-0.0003188*results_df['GHI']-0.02584*results_df['WDSP']
+1.6963*results_df['PRCP']-0.05890*results_df['SNDP']+0.05809*results_df['region']+1.9748*results_df['solar_activity']+0.03686*results_df['ONI']
results_df['PC3']=-0.00006323*results_df['slp']-0.001180*results_df['wet_bulb_temperature']-0.02384*results_df['water']-0.00002833*results_df['GHI']+0.01170*results_df['WDSP']
+0.5204*results_df['PRCP']+0.4791*results_df['SNDP']-0.004318*results_df['region']+0.008271*results_df['solar_activity']+0.8765*results_df['ONI']

# Drop columns that compose principal components
results_df = results_df.drop(['slp', 'wet_bulb_temperature', 'water', 'GHI', 'WDSP', 'PRCP', 'SNDP', 'region', 'solar_activity', 'ONI'], axis=1)

print(results_df)

print(results_df.columns)

X = []
y = []
    
for i in array:
    add_to_X = [] # create list to store each column to add to X
    new_df = results_df[results_df['file_id'] == i].drop(['file_id'], axis=1)
    add_to_y = []
    for j in range(new_df.shape[0]):
        add_to_y.append(new_df['temperatures'].iloc[j])
    y.append(add_to_y)
    #new_df = new_df.drop(['temperatures'], axis=1)
    columns_list = new_df.columns.tolist()
    for j in range(new_df.shape[0]):
        l=0
        new_row = []
        for m in columns_list:
            new_row.append(new_df.iloc[j, l])
            l += 1
        add_to_X.append(new_row)
    X.append(add_to_X)
    
#Perform k-fold cross-validation
#Taken from: https://www.geeksforgeeks.org/cross-validation-using-k-fold-with-scikit-learn/
    
#k = 5  # Number of folds
#kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
#for i, (train_index, test_index) in enumerate(kf.split(X)):
#    print(f"Fold {i}:")
#    print(f"  Training dataset index: {train_index}")
#    print(f"  Test dataset index: {test_index}")
    
#for train_indices, test_indices in kf.split(X):
#    print('Train: %s | test: %s' % (train_indices, test_indices))
    
# Create figure and axis
#fig, ax = plt.subplots(figsize=(6, 3))
#plot_kfold(kf, X, y, ax, k)
#plt.tight_layout()
#fig.subplots_adjust(right=0.6)
    
#Create train and test sets for each cross-validation split
train_X = []
train_y = []
val_X = []
val_y = []
for i in range(5):
    print(f"Fold {i+1}")
    #Add each corresponding sample for each entry of train index 
    train_X_rows = [] # Stores all the samples for one fold of train_X
    train_y_rows = [] # Stores all the samples for one fold of train_y
    for j in train_array[i]:
        train_X_rows.append(X[j])
        train_y_rows.append(y[j])
    # Stores one fold of train dataset
    train_X.append(train_X_rows)
    train_y.append(train_y_rows)
    #Add each corresponding sample for each entry of the validation index 
    val_X_rows = [] # Stores all the samples for one fold of val_X
    val_y_rows = [] # Stores all the samples for one fold of val_y
    for j in test_array[i]: 
            val_X_rows.append(X[j])
            val_y_rows.append(y[j])
    # Stores one fold of validation dataset
    val_X.append(val_X_rows)
    val_y.append(val_y_rows) 

#Convert 3D arrays to DataFrames
df_X = []
df_y = []
val_df_X = []
val_df_y = []
dataset = []
dataset_scaling = []
dataset_test = []
scaler = []
scaler_all_features = []
train = []
test = []
validation = []

for i in range(5):
    print("Fold "+str(i+1)+":")
    #Transform train_X to the correct format
    df1 = []
    dataset_df = [] # captures each weather station's dataset as values for training scaler mapping
    df_X.append(pd.DataFrame(train_X[i]))
    X_t = df_X[i].transpose()
    for k in range(84):
        X = np.array(X_t.iloc[:, k])
        df = pd.DataFrame()
        for j in range(n_time_steps):
            new_row = pd.DataFrame(X[j]).transpose()
            new_row.columns = new_df.columns
            # Add the new row
            df = pd.concat([df, new_row], ignore_index=True)
        df.columns = new_df.columns
        df1.append(df)
        dataset_df.append(df.values)
    df_X[i] = df1
    dataset.append(dataset_df)
    dataset_scaling = list(itertools.chain(*dataset)) # Holds all 84 weather station rows to train the scaling function

    #Transform train_y to the correct format
    df2 = []
    df_y.append(pd.DataFrame(train_y[i]))
    y_t = df_y[i].transpose()
    
    for j in range(84):
        y = np.array(y_t.iloc[:, j])
        y = pd.DataFrame(y)
        y.columns = ['temperatures']
        df2.append(y)
    df_y[i] = df2

    #Transform val_X to the correct format
    df3 = []
    dataset_vl_df = [] # captures each weather station's dataset as values for training scaler mapping
    val_df_X.append(pd.DataFrame(val_X[i]))
    val_X_t = val_df_X[i].transpose()
    for k in range(21):
        vl_X = np.array(val_X_t.iloc[:, k])
        vl_df = pd.DataFrame()
        for j in range(n_time_steps):
            new_row = pd.DataFrame(vl_X[j]).transpose()
            new_row.columns = new_df.columns
            # Add the new row
            vl_df = pd.concat([vl_df, new_row], ignore_index=True)
        vl_df.columns = new_df.columns
        df3.append(vl_df)
        dataset_vl_df.append(vl_df.values)
    val_df_X[i] = df3
    dataset_test.append(dataset_vl_df)    
    dataset_testing = list(itertools.chain(*dataset_test)) # Holds remaining 21 weather station rows for testing and validation

    #Transform val_y to the correct format
    df4 = []
    val_df_y.append(pd.DataFrame(train_y[i]))
    val_y_t = val_df_y[i].transpose()
    
    for j in range(21):
        v_y = np.array(val_y_t.iloc[:, j])
        v_y = pd.DataFrame(v_y)
        v_y.columns = ['temperatures']
        df4.append(v_y)
    val_df_y[i] = df4
    
    scaler.append(MinMaxScaler(feature_range=(0, 1)))
    scaler_all_features.append(MinMaxScaler(feature_range=(0, 1)))
    train.append([1])
    test.append([1])
    validation.append([1])
    
    # prepare data
    scaler[i], scaler_all_features[i], train[i] = prepare_training_data(dataset_scaling, n_lag, n_seq, n_time_steps)

    validation[i], test[i] = prepare_testing_and_validation_data(dataset_testing, n_lag, n_seq, n_time_steps, scaler_all_features[i])

    #Reshape dimensionality
    train1 = train[i]
    test1 = test[i]
    validation1 = validation[i]
    print(np.array(train1).shape)
    print(np.array(test1).shape)
    print(np.array(validation1).shape)
    train2 = []
    test2 = []
    validation2 = []

    for k in range(84):
        train2.append(train1[k][0])
        
    for k in range(11):
        test2.append(test1[k][0])

    for k in range(10):
        validation2.append(validation1[k][0])

    print(np.array(train2).shape)
    print(np.array(test2).shape)
    print(np.array(validation2).shape)

    train[i] = train2
    test[i] = test2
    validation[i] = validation2

    #Reshape dimensionality (again)
    dim_size = n_seq + 12*n_lag
    train1 = np.array(train[i]).reshape(84, dim_size)
    test1 = np.array(test[i]).reshape(11, dim_size)
    validation1 = np.array(validation[i]).reshape(10, dim_size)
    train2 = pd.DataFrame(train1).values
    test2 = pd.DataFrame(test1).values
    validation2 = pd.DataFrame(validation1).values
    print(np.array(train2).shape)
    print(np.array(test2).shape)
    print(np.array(validation2).shape)

    print(train2)

    train[i] = train2
    test[i] = test2
    validation[i] = validation2
    
print("Finished")

Model Parameters:
n_lag (number of input time steps): 165
n_seq (number of output/future prediction time steps): 60
C:\Users\User
C:\Users\User\Modeling\
             date  file_id  temperatures  specific_humidity  latitude  \
0      2006-01-31     6501     12.209677           5.386935    33.636   
1      2006-02-28     6501      8.174541           4.299929    33.636   
2      2006-03-31     6501     15.676613           6.505135    33.636   
3      2006-04-30     6501     22.464167          10.211263    33.636   
4      2006-05-31     6501     23.657258          11.737971    33.636   
...           ...      ...           ...                ...       ...   
24057  2024-07-31     9858     28.604704          15.211417    36.118   
24058  2024-08-31     9858     29.114919          15.149811    36.118   
24059  2024-09-30     9858     24.570278          10.720199    36.118   
24060  2024-10-31     9858     21.159140           6.989922    36.118   
24061  2024-11-30     9858     12.112917   

ValueError: setting an array element with a sequence.

In [ ]:
results_full_df = results_df #grab full dataset before removing last 4 years

results_df = results_df[pd.to_datetime(results_df['date']) < datetime.strptime("2021-1-1", "%Y-%m-%d")]

# Get test dataframe
results_test_df = results_full_df
results_test_df = results_test_df[pd.to_datetime(results_test_df['date']) > datetime.strptime("2020-12-31", "%Y-%m-%d")]
results_test_df = results_test_df.reset_index()

#Get one weather station
results_df = results_df[results_df['file_id'] == 6501]
results_test_df = results_test_df[results_test_df['file_id'] == 6501]

print(results_df)

# Get test values for specific humidity and temperature
specific_humidity_test_values = results_test_df['specific_humidity']
temperature_test_values = results_test_df['temperatures']

print(specific_humidity_test_values)
print(temperature_test_values)

# define a series for each column in the data frame that needs to be normalized
# Normalized columns: temperatures, specific_humidity, slp, water, region, wet_bulb_temperature,
# GHI, SNDP, latitude, longitude, elevation, Year, Month, Day, solar_activity, ONI

data = []
scaler = []
scaled_data = []

# Get Data
data.append(results_df['temperatures'])
data.append(results_df['specific_humidity'])
data.append(results_df['PC1'])
data.append(results_df['PC2'])
data.append(results_df['PC3'])

#data.append(results_df['slp'])
#data.append(results_df['water'])
#data.append(results_df['region'])
#data.append(results_df['wet_bulb_temperature'])
#data.append(results_df['GHI'])
#data.append(results_df['SNDP'])
#data.append(results_df['solar_activity'])
#data.append(results_df['ONI'])
#data.append(results_df['latitude'])
#data.append(results_df['longitude'])
#data.append(results_df['elevation'])
#data.append(results_df['Year'])
#data.append(results_df['Month'])
#data.append(results_df['Day'])

n_vars = 5 # Number of variables/predictors

# Data Preparation: Scale Data
for i in range(n_vars):
    scaler.append(StandardScaler())
    scaled_data.append([0])    
    
scaler, scaler_all_features, scaled_data = prepare_data(data)

dates = pd.date_range(start='2006-01-01', periods=180, freq='ME')

In [ ]:
# ARIMA model (for ARIMA model, all the SARIMAX model seasonal parameters are zero)

# Predict temperature

print("ARIMA Temperature Predictions:")

y = scaled_data[0] # y = temperature

for i in range(n_vars-1):
    j=i+1
    exog.append(scaled_data[j])

# Reconstruct the data frame with standardized values
data = pd.DataFrame({'y': y}, index=dates)

for i in range(n_vars-1):
    data['temp_col'] = exog[i]
    colname = 'exog' + str(i+1)
    data.rename(columns={'temp_col': colname}, inplace=True)

print("ARIMA Temperature Dataframe:")
print(data)

# Fit the ARIMA model
model_temp_ARMA = SARIMAX(data['y'], exog=None, order=(1, 0, 1), seasonal_order=(0, 0, 0, 0))
results_temp_ARMA = model_temp_ARMA.fit(disp=False)

# Print the summary of the model
print(results_temp_ARMA.summary())

# Forecasting
n_forecast = 47
forecast_temp_ARMA = results_temp_ARMA.get_forecast(steps=n_forecast, exog=exog[-n_forecast:])
forecast_temp_ARMA_mean = forecast_temp_ARMA.predicted_mean
forecast_temp_ARMA_ci = forecast_temp_ARMA.conf_int()

# Print the forecasted values
print(forecast_temp_ARMA_mean)
print(forecast_temp_ARMA_ci)

# Reshape data
forecast_temp_ARMA_mean = np.array(forecast_temp_ARMA_mean)
forecast_temp_ARMA_mean = forecast_temp_ARMA_mean.reshape(-1,1)

# Inverse transform and print forecast
inversed_temp_ARMA_mean = scaler1.inverse_transform(forecast_temp_ARMA_mean)
inversed_temp_ARMA_ci = scaler1.inverse_transform(forecast_temp_ARMA_ci)
print(inversed_temp_ARMA_mean)
print(inversed_temp_ARMA_ci)
print(temperature_test_values)

dates_predicted = pd.date_range(start='2021-01-01', periods=47, freq='ME')

combined_temp_ARMA = []
for i in range(len(temperature_test_values)):
    combined_temp_ARMA.append([dates_predicted[i], inversed_temp_ARMA_mean[i, 0], temperature_test_values[i]])

combined_temp_ARMA = pd.DataFrame(combined_temp_ARMA)
combined_temp_ARMA.columns = ['prediction_date', 'predicted_temp', 'actual_temp']

combined_temp_ARMA['error_pct'] = 100 * (combined_temp_ARMA['actual_temp'] - combined_temp_ARMA['predicted_temp'])/combined_temp_ARMA['actual_temp']

# Set display option to show all rows
pd.set_option('display.max_rows', 47)

print(combined_temp_ARMA.head(47))


In [ ]:
# SARIMAX model

print("SARIMAX Temperature Predictions:")

# Fit the SARIMAX model
model_temp_SARIMAX = SARIMAX(data['y'], exog=exog, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
results_temp_SARIMAX = model_temp_SARIMAX.fit(disp=False)

# Print the summary of the model
print(results_temp_SARIMAX.summary())

# Forecasting
n_forecast = 47
forecast_temp_SARIMAX = results_temp_SARIMAX.get_forecast(steps=n_forecast, exog=exog[-n_forecast:])
forecast_temp_SARIMAX_mean = forecast_temp_SARIMAX.predicted_mean
forecast_temp_SARIMAX_ci = forecast_temp_SARIMAX.conf_int()

# Print the forecasted values
print(forecast_temp_SARIMAX_mean)
print(forecast_temp_SARIMAX_ci)

# Reshape data
forecast_temp_SARIMAX_mean = np.array(forecast_temp_SARIMAX_mean)
forecast_temp_SARIMAX_mean = forecast_temp_SARIMAX_mean.reshape(-1,1)

# Inverse transform and print forecast
inversed_temp_SARIMAX_mean = scaler1.inverse_transform(forecast_temp_SARIMAX_mean)
inversed_temp_SARIMAX_ci = scaler1.inverse_transform(forecast_temp_SARIMAX_ci)
print(inversed_temp_SARIMAX_mean)
print(inversed_temp_SARIMAX_ci)
print(temperature_test_values)

dates_predicted = pd.date_range(start='2021-01-01', periods=47, freq='ME')

combined_temp_SARIMAX = []
for i in range(len(temperature_test_values)):
    combined_temp_SARIMAX.append([dates_predicted[i], inversed_temp_SARIMAX_mean[i, 0], temperature_test_values[i]])

combined_temp_SARIMAX = pd.DataFrame(combined_temp_SARIMAX)
combined_temp_SARIMAX.columns = ['prediction_date', 'predicted_temp', 'actual_temp']

combined_temp_SARIMAX['error_pct'] = 100 * (combined_temp_SARIMAX['actual_temp'] - combined_temp_SARIMAX['predicted_temp'])/combined_temp_SARIMAX['actual_temp']

# Set display option to show all rows
pd.set_option('display.max_rows', 47)

print(combined_temp_SARIMAX.head(47))

In [ ]:
# Predict specific humidity

# ARIMA model

print("ARIMA Specific Humidity Predictions:")

# Swap temperature and specific humidity in the dataframe
data['y'] = pd.DataFrame(scaled_data[1]) # y = specific humidity
data['exog1'] = pd.DataFrame(scaled_data[0]) # exog1 = temperature

# Define the exogenous variables
exog = (data.drop(['y'])).values

print(exog)

# Fit the SARIMAX model
model_sh_ARMA = SARIMAX(data['y'], exog=None, order=(1, 0, 1), seasonal_order=(0, 0, 0, 0))
results_sh_ARMA = model_sh_ARMA.fit(disp=False)

# Print the summary of the model
print(results_sh_ARMA.summary())

# Forecasting
n_forecast = 47
forecast_sh_ARMA = results_sh_ARMA.get_forecast(steps=n_forecast, exog=exog[-n_forecast:])
forecast_sh_ARMA_mean = forecast_sh_ARMA.predicted_mean
forecast_sh_ARMA_ci = forecast_sh_ARMA.conf_int()

# Print the forecasted values
print(forecast_sh_ARMA_mean)
print(forecast_sh_ARMA_ci)

# Reshape data
forecast_sh_ARMA_mean = np.array(forecast_sh_ARMA_mean)
forecast_sh_ARMA_mean = forecast_sh_ARMA_mean.reshape(-1,1)

# Inverse transform and print forecast
inversed_sh_ARMA_mean = scaler1.inverse_transform(forecast_sh_ARMA_mean)
inversed_sh_ARMA_ci = scaler1.inverse_transform(forecast_sh_ARMA_ci)
print(inversed_sh_ARMA_mean)
print(inversed_sh_ARMA_ci)
print(specific_humidity_test_values)

dates_predicted = pd.date_range(start='2021-01-01', periods=47, freq='ME')

combined_sh_ARMA = []
for i in range(len(specific_humidity_test_values)):
    combined_sh_ARMA.append([dates_predicted[i], inversed_sh_ARMA_mean[i, 0], specific_humidity_test_values[i]])

combined_sh_ARMA = pd.DataFrame(combined_sh_ARMA)
combined_sh_ARMA.columns = ['prediction_date', 'predicted_sh', 'actual_sh']

combined_sh_ARMA['error_pct'] = 100 * (combined_sh_ARMA['actual_sh'] - combined_sh_ARMA['predicted_sh'])/combined_sh_ARMA['actual_sh']

# Set display option to show all rows
pd.set_option('display.max_rows', 47)

print(combined_sh_ARMA.head(47))

In [ ]:
# SARIMAX model

print("SARIMAX Specific Humidity Predictions:")

# Fit the SARIMAX model
model_sh_SARIMAX = SARIMAX(data['y'], exog=exog, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
results_sh_SARIMAX = model_sh_SARIMAX.fit(disp=False)

# Print the summary of the model
print(results_sh_SARIMAX.summary())

# Forecasting
n_forecast = 47
forecast_sh_SARIMAX = results_sh_SARIMAX.get_forecast(steps=n_forecast, exog=exog[-n_forecast:])
forecast_sh_SARIMAX_mean = forecast_sh_SARIMAX.predicted_mean
forecast_sh_SARIMAX_ci = forecast_sh_SARIMAX.conf_int()

# Print the forecasted values
print(forecast_sh_SARIMAX_mean)
print(forecast_sh_SARIMAX_ci)

# Reshape data
forecast_sh_SARIMAX_mean = np.array(forecast_sh_SARIMAX_mean)
forecast_sh_SARIMAX_mean = forecast_sh_SARIMAX_mean.reshape(-1,1)

# Inverse transform and print forecast
inversed_sh_SARIMAX_mean = scaler1.inverse_transform(forecast_sh_SARIMAX_mean)
inversed_sh_SARIMAX_ci = scaler1.inverse_transform(forecast_sh_SARIMAX_ci)
print(inversed_sh_SARIMAX_mean)
print(inversed_sh_SARIMAX_ci)
print(specific_humidity_test_values)

dates_predicted = pd.date_range(start='2021-01-01', periods=47, freq='ME')

combined_sh_SARIMAX = []
for i in range(len(specific_humidity_test_values)):
    combined_sh_SARIMAX.append([dates_predicted[i], inversed_sh_SARIMAX_mean[i, 0], specific_humidity_test_values[i]])

combined_sh_SARIMAX = pd.DataFrame(combined_sh_SARIMAX)
combined_sh_SARIMAX.columns = ['prediction_date', 'predicted_sh', 'actual_sh']

combined_sh_SARIMAX['error_pct'] = 100 * (combined_sh_SARIMAX['actual_sh'] - combined_sh_SARIMAX['predicted_sh'])/combined_sh_SARIMAX['actual_sh']

# Set display option to show all rows
pd.set_option('display.max_rows', 47)

print(combined_sh_SARIMAX.head(47))